## I LEGISLATURA

### Step 1 - Preprocessing

In [1]:
import pandas as pd
import logging
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import pickle
import numpy as np
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings(action='ignore')
import time
import pprint
import pickle
import random
import numpy as np
from sklearn.metrics import accuracy_score,precision_recall_curve, average_precision_score, confusion_matrix
from scipy.sparse import csr_matrix
import matplotlib.pyplot as plt
from pprint import pprint
from datetime import datetime
import re
import json
import pandas
import operator
from functools import reduce
import editdistance
import copy
import regex
from nltk.tokenize import RegexpTokenizer
from string import punctuation
from nltk.corpus import stopwords

### Pipeline
1. Import dataset
2. Pulizia
3. Index - seduta
4. Interrogazioni
5. Presidente

In [2]:
dataset_file = 'parliament/I_legislature_italian_republic.json'

with open(dataset_file, 'r') as infile:
    dataset = json.load(infile)

In [4]:
dataset[0]['link']

'http://www.camera.it/_dati/leg01/lavori/stenografici/sed0100/sed0100.pdf'

In [6]:
len(dataset)

1113

In [15]:
from langdetect import detect, detect_langs

In [17]:
pprint(detect_langs(dataset[0]['text']))

[it:0.9999977778254913]


## Analisi e controlli

#### Studio lunghezza limite
Studio la lunghezza media dell'introduzione a ciascuna pagina in modo da impostare una thershold.

In [5]:
find = []
intro = ['Atti ', 'Alti ', 'A t t i ', 'Att i ', 'Alt i ', 'Allt ', 'At t i ', 'd t t i ']

for d in dataset:
    d['text'] = d['text'].replace('\n', '')
    year = d['date'].split('-')[0][1:4] #uso 948 perchè spesso c'è i948
    
    resulttot = []
    for i in intro:
        result = re.findall(i+'(.+?)'+year, d['text'])
        result = [i+elem+year for elem in result]
        
        resulttot.append(result)

    resulttot = reduce(operator.concat, resulttot)

    find.append(resulttot)
        

In [ ]:
find = reduce(operator.concat, find)

lenght = []
for l in find:
    if len(l)<4000:
        lenght.append(len(l))
    if len(l)>160:
        pprint(l)

plt.boxplot(lenght, vert=False)
#metto 160 come limite massimo, sopra quella soglia non considero la regex e non taglio

In [17]:
#voglio vedere quante volte riesco a trovare la dicitura che mi indica l'inizio della seduta

result = []  #lento, con search e non findall molto più veloce (tanto mi serve solo la prima volta di inizio seduta)
for d in dataset:
        res = regex.search('(La){e<2}\s+(seduta){e<3}\s+(comincia){e<4}',d['text'])
        if res != None:
            result.append(res.group())

In [18]:
pprint(len(result))
pprint(len(dataset))

#ho una buona approssimazione, quasi il 100% dei testi vengono divisi
#capita che ne trovi due ma cmq eseguo lo split solo alla prima posizione, uso quindi regex.search

1103
1113


In [7]:
#cerco quante volte riesco a trovare l'inizio delle interrogazioni

notresult = []
result = []
cont = 0

for d in dataset_pulito:
    if 'seduta' in d.keys():
        res = regex.search('(Si){e<2}\s+(dia){e<2}\s+(lettura){e<3}\s+(delle){e<2}\s+(interrogazioni){e<6}',d['seduta'])
        if res != None:
            result.append(res.group())
        else: notresult.append(cont)
    cont = cont + 1

#lo trovo solo 380 volte

In [8]:
pprint(len(result))

376


In [ ]:
#voglio vedere quante volte riesco a trovare la dicitura che indica il presidente

result = []

for d in dataset_pulito:
    if 'index' in d.keys():
        res = regex.search('(PRESIDENZA){e<4}\s+(DEL){e<3}\s+(PRESIDENTE){e<4}',d['index']) 
        
        if res != None:
            result.append(res.group())
        else:    
            res = regex.search('(PRESIDENZA){e<4}\s+(DEL){e<3}\s+(VICEPRESIDENTE){e<4}',d['index']) 

            if res != None:
                result.append(res.group())


In [42]:
pprint(len(result))
#95% delle volte

1063


## Pulizia del testo

### Prima pulizia
Rimonzione \n e introduzione di ogni pagina

In [3]:
dataset_pulito = []
find = []
intro = ['Atti ', 'Alti ', 'A t t i ', 'Att i ', 'Alt i ', 'Allt ', 'At t i ', 'd t t i ']

for d in dataset:
    d['text'] = d['text'].replace('\n', ' ')
    year = d['date'].split('-')[0][1:4] #uso 948 perchè spesso c'è i948
    
    resulttot = []
    for i in intro:
        result = re.findall(i+'(.+?)'+year, d['text'])
        result = [i+elem+year for elem in result]
        result = [elem for elem in result if len(elem)< 160]
        
        resulttot.append(result)

    resulttot = reduce(operator.concat, resulttot)
    #pprint(resulttot)
    find.append(resulttot)
    
    for l in resulttot:
        d['text'] = d['text'].replace(l, '')
        
    dataset_pulito.append(d)

In [ ]:
pprint(dataset_pulito[0])

### Split Inizio e fine seduta

In [5]:
#Divido il text in index (parte prima dell'inizio della seduta) e seduta

for d in dataset_pulito:
    res = regex.search('(La){e<2}\s+(seduta){e<3}\s+(comincia){e<4}',d['text'])
    if res !=None:
        split = d['text'].split(res.group(),1)
        d.pop('text', None)
        d['index'],d['seduta'] = split[0], split[1]
        

In [6]:
#verifico quanti documenti non sono stati splittati in index e seduta

cont = 0
for d in dataset_pulito:
    if 'index' not in d.keys():
        cont+=1
        
pprint(cont)

11


In [9]:
#Splitto la fine della seduta

for d in dataset_pulito:
    if 'seduta' in d.keys():
        res = regex.search('(La){e<2}\s+(seduta){e<2}\s+(termina){e<3}',d['seduta'])
        if res !=None:
            split = d['seduta'].split(res.group(),1)
            d['seduta'],d['end'] = split[0], split[1]

In [10]:
#verifico quanti documenti non sono stati splittati in seduta e end

cont = 0
for d in dataset_pulito:
    if 'end' not in d.keys():
        cont+=1
        
pprint(cont)

428


### Split Interrogazioni

In [11]:
#splitto e creo il campo interrogazioni

for d in dataset_pulito:
    if 'seduta' in d.keys():
        res = regex.search('(Si){e<2}\s+(dia){e<2}\s+(lettura){e<3}\s+(delle){e<2}\s+(interrogazioni){e<6}',d['seduta'])
        if res !=None:
            split = d['seduta'].split(res.group())
            d['seduta'], d['inter'] = split[0], split[1]

In [12]:
#verifico quanti documenti non sono stati splittati in seduta e interrogazioni

cont = 0
for d in dataset_pulito:
    if 'interrogazioni' not in d.keys():
        cont+=1
        
pprint(cont) #non dovrei trovarle in quanto sono già state messe in end

1113


### Trovare presidente

In [3]:
pattern = '\w+|\$[\d\.]+|\S+'
tokenizer = RegexpTokenizer(pattern)

In [23]:
def find_presidente(data):
        pres = {}
        res = regex.search('(PRESIDENZA){e<4}\s+(DEL){e<3}\s+(PRESIDENTE){e<4}',data) 
        
        if res != None:
            pres['presidente'] = data.split(res.group())[1]
            pres['presidente'] = tokenizer.tokenize(pres['presidente'])
            pres['presidente'] = [x for x in pres['presidente'] if x not in punctuation]
            return pres
        
        elif regex.search('(PRESIDENZA){e<4}\s+(DEL){e<3}\s+(VICEPRESIDENTE){e<6}',data) != None: 
            res = regex.search('(PRESIDENZA){e<4}\s+(DEL){e<3}\s+(VICEPRESIDENTE){e<6}',data)
            pres['presidente'] = data.split(res.group())[1]
            pres['presidente'] = tokenizer.tokenize(pres['presidente'])
            pres['presidente'] = [x for x in pres['presidente'] if x not in punctuation]
            return pres
        
        elif regex.search('(P R E S I D E N Z A D E L V I C E P R E S I D E N T E) {e<5}',data) != None:
            res = regex.search('(P R E S I D E N Z A D E L V I C E P R E S I D E N T E) {e<5}',data)
            pres['presidente'] = data.split(res.group())[1]
            pres['presidente'] = tokenizer.tokenize(pres['presidente'])
            pres['presidente'] = [x for x in pres['presidente'] if x not in punctuation]
            return pres
        
        elif regex.search('(P R E S I D E N Z A D E L P R E S I D E N T E) {e<5}',data) != None:
            res = regex.search('(P R E S I D E N Z A D E L P R E S I D E N T E) {e<5}',data)
            pres['presidente'] = data.split(res.group())[1]
            pres['presidente'] = tokenizer.tokenize(pres['presidente'])
            pres['presidente'] = [x for x in pres['presidente'] if x not in punctuation]
            return pres
        
        else: return False
        

In [26]:
#Vado a salvare il campo presidente
#In questa legis la maggior parte delle volte il presidente ha solo il cognome

presidenza = []
result = []
cont = 0

for d in dataset_pulito:
    if 'index' in d.keys():
        check = False
        pres = find_presidente(d['index'])
        
        if pres!= False:
            if regex.search('(PROVVISORIO){e<4}',pres['presidente'][0]) != None :
                    check = True

            if check == True:
                if pres['presidente'][2].isupper() and len(pres['presidente'][2])>1 and pres['presidente'][2][0:4]!= 'INDI':
                    pres['presidente'] = pres['presidente'][1]+" "+pres['presidente'][2]
                     
                    if pres['presidente'].split(' ')[1] == 'DEL' or pres['presidente'].split(' ')[1] == 'PAG':
                        pres['presidente'] = pres['presidente'].split(' ')[0]

                else:  
                    pres['presidente'] = pres['presidente'][1]

            else:
                if pres['presidente'][1].isupper() and len(pres['presidente'][1])>1 and pres['presidente'][1][0:4]!= 'INDI':
                    pres['presidente'] = pres['presidente'][0]+" "+pres['presidente'][1]
                    
                    if pres['presidente'].split(' ')[1] == 'DEL' or pres['presidente'].split(' ')[1] == 'PAG':
                        pres['presidente'] = pres['presidente'].split(' ')[0]
                        
                else:   pres['presidente'] = pres['presidente'][0]


            d['presidente'] = [pres['presidente']]  #salvo tutto come presidente, non tengo la distinzione

    else:
        pres = {}
        pres['presidente'] = []
   
    if pres!= False:    
        pres['doc'] = cont

    presidenza.append(pres)
    cont = cont+1

In [ ]:
pprint(presidenza)
#GRONCHI - MARTINO - LEONE

In [28]:
pickle.dump( dataset_pulito, open("Dataset_pulito.nosync/dataset_pulito_I_prima.p", "wb" ) )

In [43]:
dataset_pulito = pickle.load( open( "Dataset_pulito.nosync/dataset_pulito_I_prima.p", "rb" ) )

In [44]:
cont = 0
for d in dataset_pulito:
    if 'presidente' in d.keys():
        cont = cont+1
        
pprint(cont)

1072


In [ ]:
cont = 0
pres = {}

for d in dataset_pulito: 
    if 'seduta' in d.keys() and 'presidente' in d.keys():
        d['seduta'] = [d['seduta']]
        i = 0
        res = None
        
        r = regex.search('(PRESIDENZA){e<4}\s+(DEL){e<3}\s+(PRESIDENTE){e<6}',d['seduta'][0])
        if r!= None:
            res = r

        r = regex.search('(PRESIDENZA){e<4}\s+(DEL){e<3}\s+(VICEPRESIDENTE){e<6}',d['seduta'][0])
        if r != None and res!= None and r.span()[0]<res.span()[0]:
            res = r

        r = regex.search('(P R E S I D E N Z A D E L V I C E P R E S I D E N T E) {e<5}',d['seduta'][0])
        if r != None and res!= None and r.span()[0]<res.span()[0]:
            res = r    
            
        r = regex.search('(P R E S I D E N Z A D E L P R E S I D E N T E) {e<5}',d['seduta'][0])
        if r != None and res!= None and r.span()[0]<res.span()[0]:
            res = r 

        while res != None: #continuo a splittare finchè trovo che c'è un nuovo presidente
            check = False
            split = d['seduta'][i].split(res.group(),1)
            d['seduta'].pop(i)
            d['seduta'].append(split[0])
            d['seduta'].append(split[1])

            pres['presidente'] = tokenizer.tokenize(split[1])
            pres['presidente'] = [x for x in pres['presidente'] if x not in punctuation]

            if regex.search('(PROVVISORIO){e<4}',pres['presidente'][0]) != None :
                check = True
                
            if check == True:
                pres['presidente'] = pres['presidente'][1]

            else:
                pres['presidente'] = pres['presidente'][0]

            d['presidente'].append(pres['presidente'])
            
            i = i +1
            
            res = None

            r = regex.search('(PRESIDENZA){e<4}\s+(DEL){e<3}\s+(PRESIDENTE){e<6}',d['seduta'][i])
            if r!= None:
                res = r

            r = regex.search('(PRESIDENZA){e<4}\s+(DEL){e<3}\s+(VICEPRESIDENTE){e<6}',d['seduta'][i])
            if r != None and res!= None and r.span()[0]<res.span()[0]:
                res = r

            r = regex.search('(P R E S I D E N Z A D E L V I C E P R E S I D E N T E) {e<5}',d['seduta'][i])
            if r != None and res!= None and r.span()[0]<res.span()[0]:
                res = r    
            
            r = regex.search('(P R E S I D E N Z A D E L P R E S I D E N T E) {e<5}',d['seduta'][i])
            if r != None and res!= None and r.span()[0]<res.span()[0]:
                res = r 
            
    pprint(cont)       
    cont = cont+1

In [ ]:
cont = 0
res = []
for d in dataset_pulito:
    if 'presidente' in d.keys():
        print(cont)
        print(d['presidente'])
        res.append(cont)
        
    cont = cont+1

In [42]:
pprint(dataset_pulito[75]['link'])

'http://www.camera.it/_dati/leg01/lavori/stenografici/sed0099/sed0099.pdf'


In [48]:
len_p = []
for d in dataset_pulito:
    if 'presidente' in d.keys():
        for p in d['presidente']:
             len_p.append(len(d['presidente']))

In [49]:
pprint(np.max(len_p))
pprint(np.mean(len_p))
pprint(np.median(len_p))

6
2.0519978106185004
2.0


In [50]:
pickle.dump( dataset_pulito, open("Dataset_pulito.nosync/dataset_pulito_I.p", "wb" ) )

## Divisione dei discorsi

In [4]:
dataset_pulito = pickle.load( open( "Dataset_pulito.nosync/dataset_pulito_I.p", "rb" ) )

In [5]:
legis_I_leaders = {'DC': [('Aldo', 'Moro'), ('Oscar Luigi', 'Scalfaro'), ('Alcide', 'De Gasperi'), ('Amintore', 'Fanfani'), 
                          ('Giovanni', 'Gronchi'), ('Antonio', 'Segni'), ('Giovanni', 'Leone'), ('Mariano', 'Rumor'), 
                          ('Giulio', 'Andreotti')],
'PCI': [('Palmiro', 'Togliatti'), ('Giorgio','Amendola'), ('Giuseppe', 'Di Vittorio'), ('Antonio', 'Giolitti'), ('Pietro', 'Ingrao'), 
        ('Nilde', 'Iotti'), ('Giancarlo', 'Pajetta'), ('Luigi', 'Longo')],
'PSI': [('Pietro', 'Nenni'), ('Carlo', 'Matteotti'), ('Lelio', 'Basso'), ('Giacomo','Mancini'), ('Francesco', 'De Martino')],
'PSDI': [('Piero', 'Calamandrei'), ('Giuseppe', 'Saragat')],
'PLI': ('Gaetano', 'Martino'),
'PRI': [('Ugo', 'La Malfa'), ('Randolfo','Pacciardi')],
'MISTO' : [('Giorgio', 'Almirante'), ('Guido', 'Russo Perez')],
'MONARCHICO': ('Alfredo', 'Covelli')}

politici_max_def_name = []

for k in legis_I_leaders.keys():
    if type(legis_I_leaders[k]) == list:
        for t in legis_I_leaders[k]:
            if t[1] == 'Pajetta':
                politici_max_def_name.append(t[1].upper()+" "+t[0].upper())
            else:
                politici_max_def_name.append(t[1].upper())
    else: politici_max_def_name.append(legis_I_leaders[k][1].upper())

In [ ]:
pprint(politici_max_def_name)

In [7]:
politici = {}

for d in dataset_pulito:
    if 'presidente' in d.keys():
        for p in d['presidente']:
            if p not in politici.keys():
                politici[p] = 1

In [ ]:
pprint(politici)

In [ ]:

cont = 0
flag = 0

for data in dataset_pulito:
    if 'seduta' in data.keys() and 'presidente' in data.keys():
        for k in range(0,len(data['seduta'])):
            presidente = data['presidente'][k]
            
            text = data['seduta'][k]
            text_token = tokenizer.tokenize(text)
            text_token = [x for x in text_token if x not in punctuation]

            for i in range(0,len(text_token)):

                    word = text_token[i]
                    politico = re.search(r"[A-Z]{2,}", word)

                    if politico != None and i+2<len(text_token):
                        if editdistance.eval(politico.group(), 'PRESIDENTE')<4:
                            pol = presidente
                            politici[pol] = politici[pol]+1
                        
                        else:
                            if flag == 0: 
                                flag = 0
                                if text_token[i+1].isupper() and len(text_token[i+1])>1 and text_token[i+1][0]!='(' : #ignoro cose tipo XIII legisl

                                    if text_token[i+2].isupper() and len(text_token[i+2])>2 and text_token[i+2][0]!='(':
                                        pol = word+" "+text_token[i+1]+" "+text_token[i+2]

                                        if pol not in politici.keys():
                                            politici[pol] = 1
                                        else: politici[pol] = politici[pol]+1
                                        flag = 2


                                    else:
                                        pol = word+" "+text_token[i+1]
                                        if pol not in politici.keys():
                                            politici[pol] = 1
                                        else: politici[pol] = politici[pol]+1
                                        flag = 1
                                    
                                else:
                                    pol = word
                                    if pol not in politici.keys():
                                            politici[pol] = 1
                                    else: politici[pol] = politici[pol]+1
                                        
                            else: flag = flag -1

            cont = cont +1

In [ ]:
pprint((politici))

In [11]:
politici_max = [] #tengo solamente quelli con almeno 5 occorrenze

for k in politici.keys():
    if politici[k] > 4:
        politici_max.append([k, politici[k]])

In [12]:
pprint(len(politici_max))
#li considero come tutti i politici
#lo uso per scartare quelli che non dovrebbero essere politici

1959


In [13]:
politici_max_name = [elem[0] for elem in politici_max]

In [14]:
#li aggrego in base alle somiglianze nel nome

politici_definitivi = []
cont = 1

for p in politici_max:
    for i in range(cont, len(politici_max)):
        if len(p[0]) <=  10 and editdistance.eval(p[0],politici_max[i][0])<3:
            p[1] = p[1] + politici_max[i][1]
            politici_max[i][1] = 0
        
        if len(p[0]) > 10 and editdistance.eval(p[0],politici_max[i][0])<4:
            p[1] = p[1] + politici_max[i][1]
            politici_max[i][1] = 0
            
    cont = cont +1
    

In [15]:
#elimino quelli che sono stati aggregati
politici_max_agg = [elem for elem in politici_max if elem[1]>0]

In [ ]:
pprint(sorted(politici_max_agg))

In [ ]:
# non ho bisono della soglia in quanto ho già i politici di interesse, capire cosa usare per decidere se è 
#un politico o no

In [17]:
def check_politico(pol):
    
    if politico == 'SCOTTI': #Scotti crea problemi con IOTTI, lo ignoro in quanto non mi interessa
        return False, None
    
    if politico == 'CAVALLI': #Cavalli crea problemi con Covelli
        return False, None
    
    if politico == 'PAJETTA GIAN':
        return True, 'PAJETTA GIANCARLO'
        
    for cog in politici_max_def_name:
            res = editdistance.eval(cog,pol)
            
            if len(pol)<=5 and res <2:
                return True, cog
            
            elif len(pol)  <= 10 and res<3: #così non posso fare de martino = martino
                return True, cog
                
            elif len(pol) > 10 and res<5:
                return True, cog
    
    return False, None

In [18]:
discorsi = {}

for cog in politici_max_def_name:
    discorsi[cog] = []
    
    #in questo caso ho solo cognomi corretti, ma con i confronti dovrei riuscire a recuperarne la maggior parte

In [ ]:
#Salvo i discorsi

cont = 0
flag = 0
d = {}


for data in dataset_pulito:
    if 'seduta' in data.keys() and 'presidente' in data.keys():
         for k in range(0,len(data['seduta'])):
            presidente = data['presidente'][k]
            
            text = data['seduta'][k]
            text_token = tokenizer.tokenize(text)
            text_token = [x for x in text_token if x not in punctuation]
            i = 0
            flag = 0
            d = {}

            for word in text_token:

                    politico = re.search(r"[A-Z]{2,}", word) #2 perchè ho de martino e de gasperi

                    if politico != None and i+2<len(text_token):
                        
                        if editdistance.eval(politico.group(), 'PRESIDENTE')<4:
                            pprint(politico.group())
                            
                            if d!={} and len(d['discorso'])>2:
                                discorsi[d['politico']].append(d['discorso'])
                                pprint(str(cont)+' '+d['politico'])
                            
                                
                            flag = 0
                            d = {}
                            
                            pres, cog_pres = check_politico(presidente) #controllo se il presidente mi interessa
                            
                            if pres == True:
                                d['politico'] = cog_pres
                                d['discorso'] = []
                                flag = 1
                        
                        else:
                            pol = politico.group()
      
                            pol2 =  politico.group()+" "+text_token[i+1]
                            
                            if editdistance.eval('LEONE-MARCHESANO', word)<4:
                                pol = word #la regex non riconosce -
        
                            if text_token[i+1] == '-':
                                pol2 = politico.group()+" "+text_token[i+1]+" "+text_token[i+2]
                            
                            if (pol in politici_max_name) or (pol2 in politici_max_name): #se non è in questa lista non lo considero un politico 
                                    if d!={} and len(d['discorso'])>2: #salvo perchè significa che inizia un altro discorso
                                        discorsi[d['politico']].append(d['discorso'])
                                        pprint(str(cont)+' '+d['politico'])
                                        
                                    
                                    flag = 0
                                    d = {}
                                    
                                        

                                    if (text_token[i+1].isupper() and len(text_token[i+1])>2) or text_token[i+1][0]=='-': 
                                        check, cog = check_politico(pol2)
                                        if check == True:
                                            d['politico'] = cog
                                            d['discorso'] = []
                                            text_token.pop(i+1)
                                            
                                            if text_token[i+1] == '-':
                                                text_token.pop(i+1)
                                            flag = 1
                                            
                                    elif editdistance.eval(word, 'PAJETTA')<3:
                                        pol3 = politico.group()+" "+text_token[i+1]+" "+text_token[i+2]
                                        check, cog = check_politico(pol3)
                                        if check == True:
                                            d['politico'] = cog
                                            d['discorso'] = []
                                            text_token.pop(i+1)
                                            text_token.pop(i+1)
                                            
                                            flag = 1
                                        

                                    else:
                                        check, cog = check_politico(pol)
                                        if check == True:
                                            flag = 1
                                            d['politico'] = cog
                                            d['discorso'] = []
                                            

                            elif flag == 1: #vuol dire che è una parola in maiuscolo ma non è un politico
                                d['discorso'].append(word)
                         
                    elif flag ==1:
                        d['discorso'].append(word)
                    i = i +1
                    
            if d != {} and len(d['discorso'])>2: 
                discorsi[d['politico']].append(d['discorso'])
                pprint(str(cont)+' '+d['politico'])
               
    cont = cont +1

In [21]:
print(discorsi['DI VITTORIO'][0])

['Com', "'e", 'il', 'lodo', 'De', 'Gasperi']


In [134]:
pprint(dataset_pulito[10]['link'])

'http://www.camera.it/_dati/leg01/lavori/stenografici/sed0013/sed0013.pdf'


In [20]:
for k in sorted(discorsi.keys()):
    print(k+" "+str(len(discorsi[k])))

ALMIRANTE 680
AMENDOLA 24
ANDREOTTI 225
BASSO 413
CALAMANDREI 97
COVELLI 384
DE GASPERI 563
DE MARTINO 234
DI VITTORIO 1456
FANFANI 814
GIOLITTI 883
GRONCHI 16892
INGRAO 57
IOTTI 137
LA MALFA 418
LEONE 3917
LONGO 93
MANCINI 197
MARTINO 4668
MATTEOTTI 0
MORO 1648
NENNI 13
PACCIARDI 415
PAJETTA GIANCARLO 23
RUMOR 105
RUSSO PEREZ 582
SARAGAT 202
SCALFARO 242
SEGNI 1331
TOGLIATTI 350


In [133]:
for k in sorted(discorsi.keys()):
    print(k+" "+str(len(discorsi[k])))

ALMIRANTE 680
AMENDOLA 24
ANDREOTTI 225
BASSO 413
CALAMANDREI 97
COVELLI 384
DE GASPERI 563
DE MARTINO 234
DI VITTORIO 1456
FANFANI 814
GIOLITTI 883
GRONCHI 16892
INGRAO 57
IOTTI 137
LA MALFA 418
LEONE 3917
LONGO 93
MANCINI 197
MARTINO 4668
MATTEOTTI 0
MORO 1648
NENNI 13
PACCIARDI 415
PAJETTA GIANCARLO 5
RUMOR 105
RUSSO PEREZ 582
SARAGAT 202
SCALFARO 242
SEGNI 1331
TOGLIATTI 350


In [ ]:
pprint(dataset_pulito[0])

In [143]:
leng = []

for k in discorsi.keys():
    if k!= 'GRONCHI':
        leng.append(len(discorsi[k]))

In [145]:
print(leng)
print(np.mean(leng))

[1648, 242, 563, 814, 1331, 3917, 105, 225, 350, 24, 1456, 883, 57, 137, 5, 93, 13, 0, 413, 197, 234, 97, 202, 4668, 418, 415, 680, 582, 384]
694.9310344827586


In [22]:
#Salvo un file per ciascun politico con i suoi discorso

for cog in discorsi.keys():
    pickle.dump( discorsi[cog], open( 'Discorsi-I-legis/'+cog+".p", "wb" ) )

In [23]:
pickle.dump( list(discorsi.keys()), open( 'Discorsi-I-legis/cognomi_discorsi.p', "wb" ) ) 

### Punto della situazione
#### Cose fatte:
1. Rimosso intestazione
2. Trovato "la seduta comincia" e splittato in index e seduta
3. Trovato il presidente
4. Trovato "Si dia lettura delle interrogazioni" e splittato in seduta e inter
5. Salvato tutto come dataset_pulito
6. Rimosso la punteggiatura e identificati i politici e relativi discorsi
7. Salvato tutto in un file per ciascun politico
 
                                                
30 politici di interesse - 1234 discorsi in media (694 senza Gronchi) con mediana 367 